In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy.signal
import numpy as np
import seaborn as sns

In [2]:
plt.rcParams["figure.figsize"]=[15,5]

In [3]:
raw_pp=pd.read_csv('pickandplace1_vibration.csv', usecols=['timestamp', 'data.ax', 'data.ay', 'data.az'])
raw_pp.dropna(axis=1, inplace=True)
raw_pp['timestamp'] =  pd.to_datetime(raw_pp['timestamp'])

In [4]:
#Load LD data and plot
raw_pp["acc"] = ( raw_pp["data.ax"]**2 + raw_pp["data.az"]**2) ** 0.5
acc=raw_pp[["timestamp", "acc"]]
acc.index = acc.timestamp
test_pp=acc
# test_pp['acc_rolling_sum']=pd.Series.to_frame(test_pp.acc.rolling(100*10, center=True).sum())
test_pp['acc_rolling_std']=pd.Series.to_frame(test_pp.acc.rolling(100*2, center=True).std())


/home/raghu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [ ]:
%matplotlib qt

In [ ]:
# plt.plot(test_pp['timestamp'], test_pp['acc'])
# plt.plot(test_pp['timestamp'], test_pp['acc_rolling_std'])

In [ ]:
test_pp['detections']=pd.Series.to_frame(test_pp.acc_rolling_std.rolling(100, center=True).sum())

/home/raghu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [ ]:
plt.plot(test_pp['timestamp'], test_pp['acc']*100)
plt.plot(test_pp['timestamp'], test_pp['detections'])

In [ ]:
plt.plot(test_pp['timestamp'], test_pp['detections'])

In [ ]:
test_pp["boards"]=0
test_pp["boards"][test_pp['detections']>=2] = 1

In [ ]:
plt.plot(test_pp["timestamp"],test_pp["boards"])

In [ ]:
working_times_raw=scipy.signal.find_peaks(test_pp.boards, distance=50*72, width=1)

In [ ]:
# plt.plot(test_pp.index, test_pp.boards)
# plt.stem(working_times_raw[1]['right_ips'], np.ones(len(working_times_raw[0])), 'r')

In [ ]:
sns.distplot(working_times_raw[1]['widths']/72)

In [ ]:
test_pp.iloc[-1]['timestamp'] - test_pp.iloc[0]['timestamp']

In [ ]:
board_times_raw=pd.Series(working_times_raw[1]['widths'])/72

In [ ]:
board_times_raw.mode().mean()

In [ ]:
# printing_delays[1]['widths']
q1 = np.percentile(board_times_raw, 5)
q3 = np.percentile(board_times_raw, 95)

#board_times = [i for i in board_times_raw if i > q1 and i < q3]

board_times = board_times_raw[board_times_raw.between(q1,q3)]
# printing_delays = filter(lambda x: x)

In [ ]:
sns.distplot(board_times, bins=30, color='chocolate')

In [ ]:
board_times.mode().mean()

In [ ]:
print('Number of Boards deteced: 'board_times.shape[0])